In [15]:
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(monocle3))
suppressPackageStartupMessages(library(ggplot2))
set.seed(42)

In [ ]:
# Files directory

monocole_3_learned_object_RDS_direction <- "/home/arsham79/scratch/nsclc/results/monocole3_cds_objectd_trained.rds"
monocole_3_DEGs_RDS_direction <- "/home/arsham79/scratch/nsclc/results/DEGs_monocole3.rds"

In [2]:
cds <- readRDS(monocole_3_learned_object_RDS_direction)

In [3]:
plot_cells(cds, color_cells_by = "partition", label_branch_points=FALSE, label_leaves=FALSE)

ERROR: Error in plot_cells(cds, color_cells_by = "partition", label_branch_points = FALSE, : could not find function "plot_cells"


In [ ]:
plot_cells(cds, color_cells_by = "cell_type", label_groups_by_cluster=FALSE, label_leaves=FALSE, label_branch_points=FALSE, labels_per_group = FALSE)

In [ ]:
plot_cells(cds, color_cells_by = "pseudotime", label_groups_by_cluster=FALSE, label_leaves=FALSE, label_branch_points=TRUE)

In [ ]:
cds$pseudotime_value <- pseudotime(cds)
pseudotime_df <- as.data.frame(colData(cds))

In [ ]:
ggplot(pseudotime_df, aes(x = pseudotime_value, y = reorder(cell_type, pseudotime_value, median), fill = cell_type)) + geom_boxplot() + NoLegend()

In [ ]:
degs <- readRDS(monocole_3_DEGs_RDS_direction)